# Робота з моделями Mistral

## Вступ

У цьому уроці розглянемо:
- Огляд різних моделей Mistral
- Розуміння випадків використання та сценаріїв для кожної моделі
- Приклади коду, які демонструють унікальні можливості кожної моделі.


## Моделі Mistral

У цьому уроці ми розглянемо три різні моделі Mistral: **Mistral Large**, **Mistral Small** та **Mistral Nemo**.

Кожна з цих моделей доступна безкоштовно на маркетплейсі моделей Github. Код у цьому ноутбуці буде використовувати ці моделі для запуску коду. Більше інформації про використання Github Models для [прототипування з AI-моделями](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 наразі є флагманською моделлю від Mistral і розроблена для корпоративного використання.

Модель є оновленням оригінальної Mistral Large і пропонує:
- Більше контекстне вікно — 128k проти 32k
- Кращу продуктивність у завданнях з математики та програмування — середня точність 76,9% проти 60,4%
- Покращену багатомовну підтримку — підтримувані мови: англійська, французька, німецька, іспанська, італійська, португальська, нідерландська, російська, китайська, японська, корейська, арабська та гінді.

Завдяки цим можливостям, Mistral Large відмінно підходить для:
- *Генерації з підкріпленням пошуком (RAG)* — завдяки більшому контекстному вікну
- *Виклику функцій* — ця модель має вбудовану підтримку виклику функцій, що дозволяє інтегруватися із зовнішніми інструментами та API. Виклики можна здійснювати як паралельно, так і послідовно, один за одним.
- *Генерації коду* — ця модель чудово справляється з генерацією коду на Python, Java, TypeScript та C++.


У цьому прикладі ми використовуємо Mistral Large 2 для запуску патерну RAG над текстовим документом. Питання написане корейською мовою і стосується діяльності автора до вступу в університет.

Використовується модель Cohere Embeddings для створення ембеддінгів текстового документа, а також самого питання. Для цього прикладу використовується пакет faiss для Python як векторне сховище.

Підказка, яку надсилають моделі Mistral, містить як питання, так і знайдені фрагменти, схожі на питання. Модель потім надає відповідь природною мовою.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small — це ще одна модель із сімейства Mistral, яка належить до преміальної/корпоративної категорії. Як випливає з назви, ця модель є малою мовною моделлю (SLM). Переваги використання Mistral Small такі:
- Економія коштів у порівнянні з LLM Mistral, такими як Mistral Large та NeMo — зниження ціни на 80%
- Низька затримка — швидша відповідь у порівнянні з LLM Mistral
- Гнучкість — може розгортатися в різних середовищах з меншими обмеженнями щодо необхідних ресурсів

Mistral Small чудово підходить для:
- Текстових завдань, таких як узагальнення, аналіз настроїв і переклад
- Застосунків, де часто надходять запити, завдяки своїй економічності
- Завдань із низькою затримкою, пов’язаних із кодом, наприклад, рецензування та пропозиції коду


## Порівняння Mistral Small та Mistral Large

Щоб побачити різницю в затримці між Mistral Small і Large, виконайте наведені нижче команди.

Ви помітите різницю у часі відповіді — приблизно 3-5 секунд. Зверніть увагу також на довжину та стиль відповідей на однаковий запит.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Порівняно з двома іншими моделями, розглянутими в цьому уроці, Mistral NeMo — єдина безкоштовна модель з ліцензією Apache2.

Її вважають покращеною версією попередньої відкритої LLM від Mistral — Mistral 7B.

Деякі інші особливості моделі NeMo:

- *Більш ефективна токенізація:* У цій моделі використовується токенізатор Tekken замість більш поширеного tiktoken. Це забезпечує кращу продуктивність для більшої кількості мов і коду.

- *Донавчання:* Базова модель доступна для донавчання. Це дає більше гнучкості для сценаріїв, де може знадобитися донавчання.

- *Вбудований виклик функцій* — Як і Mistral Large, ця модель навчена на виклику функцій. Це робить її унікальною, адже вона одна з перших відкритих моделей, які це вміють.


## Mistral NeMo

Порівняно з двома іншими моделями, розглянутими в цьому уроці, Mistral NeMo — єдина безкоштовна модель з ліцензією Apache2.

Її вважають покращеною версією попередньої відкритої LLM від Mistral, Mistral 7B.

Деякі інші особливості моделі NeMo:

- *Більш ефективна токенізація:* У цій моделі використовується токенізатор Tekken замість більш поширеного tiktoken. Це забезпечує кращу продуктивність для більшої кількості мов і коду.

- *Донавчання:* Базова модель доступна для донавчання. Це дає більше гнучкості для випадків використання, де може знадобитися донавчання.

- *Вбудований виклик функцій* — Як і Mistral Large, ця модель навчена на виклику функцій. Це робить її унікальною, адже вона є однією з перших відкритих моделей з такою можливістю.


### Порівняння токенізаторів

У цьому прикладі ми розглянемо, як Mistral NeMo виконує токенізацію у порівнянні з Mistral Large.

Обидва приклади використовують однаковий запит, але ви помітите, що NeMo повертає менше токенів, ніж Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Навчання не закінчується тут, продовжуйте свій шлях

Після завершення цього уроку перегляньте нашу [колекцію навчальних матеріалів з генеративного ШІ](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), щоб і далі поглиблювати свої знання у сфері генеративного штучного інтелекту!



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
